# Assignment 2

#### First Name: Flàvia 
#### Last Name: Ferrús Marimón


## 1. Load Data from JSON

In [1]:

from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Twitter Analysis")\
.getOrCreate()


In [8]:
df_twitter = spark.read.json("corona_tweet_new.json")

In [9]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [15]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)


In [14]:
from pyspark.sql.functions import col

# Select only the required columns from the nested user struct column
user_cols = ["id_str", "followers_count", "friends_count", "created_at"]
user_sub_df = df_twitter.select([col("user."+c).alias("user_"+c) for c in user_cols])

# Join the original DataFrame with the extracted user sub-DataFrame
df_twitter1 = df_twitter.join(user_sub_df, how="left")

# Drop the original nested user column
df_twitter1 = df_twitter1.drop("user")

# Print the resulting schema
df_twitter1.printSchema()


root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [16]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter1.count()

252619236

In [18]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regular expresion and the group would be 1 for this regular expression.
#(4 points)


In [17]:
from pyspark.sql.functions import regexp_extract, col

# Extract the source label from the source column using a regular expression
df_twitter2 = df_twitter1.withColumn("extracted_source", regexp_extract(col("source"), '<a [^>]+>([^<]+)', 1))

# Select the extracted_source and source columns and show the resulting DataFrame
df_twitter2.select(col('extracted_source'), col('source')).show()


+----------------+--------------------+
|extracted_source|              source|
+----------------+--------------------+
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
+----------------+--------------------+
only showing top 20 rows



In [44]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regular expresion and the group would be 1 for this regular expression.
#(4 points)
from pyspark.sql.functions import regexp_extract, col

df_twitter=df_twitter.<FILL_IN>
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
+----------

In [21]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter2.rdd.map(tuple)

In [22]:
# Create a temporay table in memory with name as twitter (1 point)
df_twitter2.createOrReplaceTempView("twitter")


In [23]:
rdd_twitter.take(2)

[('Mon Apr 20 15:01:57 +0000 2020',
  5545,
  [],
  '1252251164227362821',
  None,
  None,
  'India',
  3460,
  5477,
  '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  '93612837',
  121,
  759,
  'Mon Nov 30 11:38:08 +0000 2009',
  'Twitter Web App'),
 ('Mon Apr 20 15:01:57 +0000 2020',
  5545,
  [],
  '1252251164227362821',
  None,
  None,
  'India',
  3460,
  5477,
  '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  '346443880',
  208,
  1196,
  'Mon Aug 01 08:15:42 +0000 2011',
  'Twitter Web App')]

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

In [25]:
# Using RDD

## We need to get the total number of rows with unique user_id_str

import pyspark
#sc = pyspark.SparkContext('local[*]')
# Display the type of the Spark Context sc
#type(sc)
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Big Data Assisgnment 2") \
    .getOrCreate()
sc=spark.sparkContext

In [35]:
## Find the index of the column where we have the user id
rdd_twitter.first()[10]

## This is the id 

'93612837'

In [ ]:
#unique_users = rdd_twitter.map(lambda x: x[10]).distinct().count()
#print(unique_users)

Recall that this code takes a lot of memory, since the function `distinct` is not the optimum way to compute the unique number of users from the dataset. 

Observe that we can get the unique number of users by applying an alternative `rdd` code in spark

In [ ]:
users = rdd_twitter.map(lambda x: (x.user_id_str, 1))

unique_users2 = users.reduceByKey(lambda a, b : a+b)
## Could have used distinct()
uniqueUserCount = unique_users2.count()
print ('Unique hosts: %d' % uniqueUserCount)

In [ ]:
# Using DataFrame
unique_users_df = df_twitter2.select("user_id_str").distinct().count()
print("Total number of unique users: ", unique_users_df)

In [ ]:
# Using spark.sql and the temporay table.

## We use the table we have created, this is:

#df_twitter2.createOrReplaceTempView("twitter")

# Count the total number of unique users
unique_users_sql = spark.sql("SELECT COUNT(DISTINCT user_id_str) FROM twitter")

# Print the result
print(unique_users_sql.first()[0])


#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [51]:
# Using RDD

In [52]:
# Using DataFrame

In [53]:
# Using spark.sql and the temporay table.

#### 2.3 Get total number unique extracted_source (1 point each)

In [54]:
# Using RDD

In [55]:
# Using DataFrame

In [56]:
# Using spark.sql and the temporay table.

#### 2.4 Get top 5 most used extracted_source

In [57]:
# Using RDD (5 points)

In [58]:
# Using DataFrame (2 points)

In [59]:
# Using spark.sql and the temporay table. (2 points)

#### 2.5 Get count of distinct hastags used ( 5 point each) 

In [60]:
# Using RDD

In [61]:
# Using DataFrame

In [62]:
# Using spark.sql and the temporay table.

#### 2.6 Get top 5 hashtags

In [63]:
# Using RDD (4 points)

In [64]:
# Using DataFrame (2 points)

In [84]:
# Using spark.sql and the temporay table. (2 points)

#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [85]:
# Using RDD

In [86]:
# Using DataFrame


In [87]:
# Using spark.sql and the temporay table.


#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [88]:
# Using RDD

In [89]:
# Using DataFrame

In [90]:
# Using spark.sql and the temporay table.

#### 2.9 Total number of tweets per country

In [91]:
# Using RDD (3 points)

In [92]:
# Using DataFrame (2 points)

In [93]:
# Using spark.sql and the temporay table. (1 point)

## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [94]:
# Using DataFrame

#### 3.2 Save the data as parquet files (1 points)

In [95]:
# Using DataFrame